## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [40]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [47]:
# 讀取digits資料集
digits= datasets.load_digits()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=4)

# 建立模型
clf = GradientBoostingClassifier()

# 訓練模型
clf.fit(x_train, y_train)

# 預測測試集
y_pred = clf.predict(x_test)
print(y_pred)

[8 7 0 5 3 5 1 3 1 8 2 7 8 4 7 7 8 3 0 6 9 7 8 0 8 6 8 1 0 0 0 2 7 0 1 7 6
 3 1 3 4 2 9 1 2 0 0 7 3 3 2 9 7 6 1 8 5 8 6 7 5 6 9 3 3 4 1 9 7 8 4 5 0 4
 1 6 6 7 8 1 2 6 9 1 7 4 2 6 7 3 7 5 4 8 5 1 5 6 7 1 2 5 5 2 0 8 5 2 2 3 0
 4 5 6 9 3 9 5 7 4 7 8 9 4 9 7 9 7 9 4 3 0 5 4 9 2 8 2 9 6 2 6 0 5 5 8 9 2
 5 3 4 4 2 0 9 8 4 3 6 6 2 9 7 1 5 7 6 0 5 3 2 3 1 3 2 6 6 0 8 2 5 7 6 8 4
 6 2 2 0 4 0 3 0 4 0 1 5 6 4 7 1 5 4 5 5 3 4 4 6 3 7 1 1 3 5 7 5 0 1 9 5 0
 8 7 4 0 6 6 5 0 2 4 2 9 4 0 6 2 5 1 9 6 3 9 0 8 3 1 2 1 3 2 0 9 0 7 5 9 1
 8 6 9 6 8 8 6 2 4 5 9 9 1 5 2 8 4 7 9 8 8 0 1 7 3 2 2 1 0 3 2 3 9 7 2 0 0
 1 2 6 0 9 9 7 8 5 4 0 0 1 5 7 1 0 3 9 8 5 4 7 0 4 9 5 6 0 8 2 0 5 2 3 2 2
 4 2 8 7 5 8 8 6 9 2 6 4 5 9 5 4 1 7 1 7 3 4 8 5 4 3 7 7 5 6 3 7 8 6 0 3 5
 2 3 8 0 0 7 8 3 9 0 0 2 2 9 7 4 3 9 3 1 2 9 1 0 1 3 9 8 9 9 5 5 0 1 5 4 4
 9 2 8 1 6 5 4 6 3 8 3 9 1 2 8 9 1 7 5 7 9 8 9 3 8 0 2 5 7 7 9 8 8 2 5 9 0
 4 7 9 8 7 5]


In [48]:
acc = metrics.accuracy_score(y_test, y_pred)
print("Acuuracy: ", acc)

Acuuracy:  0.9688888888888889


In [53]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300, 400]
max_depth = [1, 3, 5, 7]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid,cv=10, scoring="accuracy", n_jobs=-1, verbose=2)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 10-fold cross-validadtion，總共 16 種參數組合，總共要 train 160 次模型

Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.6min finished


In [54]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.960653 using {'n_estimators': 400, 'max_depth': 3}


In [55]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)
print(y_pred)

[8 7 0 5 3 5 1 3 1 8 2 7 8 4 7 7 8 3 0 6 9 7 8 0 8 6 8 1 0 0 0 2 7 0 1 7 6
 3 1 3 4 2 9 1 2 0 0 7 3 3 2 9 7 6 1 8 5 8 6 7 5 6 9 3 3 4 1 9 7 8 4 5 0 4
 1 6 6 7 8 1 2 6 9 1 7 4 2 6 7 3 7 5 4 8 5 1 5 6 7 1 2 5 5 2 0 8 5 2 2 3 0
 4 5 6 9 3 9 5 7 4 7 8 9 4 9 7 9 7 9 4 3 0 5 4 9 2 6 2 9 6 2 6 0 5 5 8 9 2
 5 3 4 4 2 0 9 8 4 3 6 6 2 9 7 1 5 7 6 0 5 3 2 3 1 3 2 6 6 0 8 2 5 7 6 8 4
 6 2 2 0 4 0 3 0 4 0 1 5 6 4 7 1 5 4 5 5 3 4 4 6 3 7 1 1 3 5 7 5 0 1 9 5 0
 8 7 4 0 6 6 5 0 2 4 2 9 4 0 6 2 5 1 9 6 3 9 0 8 3 1 2 1 3 2 0 9 0 7 5 9 1
 8 6 9 6 8 8 6 2 4 5 9 9 1 5 2 8 4 7 9 8 8 0 1 7 3 2 2 1 0 3 2 3 9 7 2 0 0
 1 2 6 0 9 9 7 8 5 4 0 0 1 5 7 1 0 3 9 8 5 4 7 0 4 9 5 6 0 8 2 0 5 2 3 2 2
 4 2 8 7 5 8 8 6 9 2 6 4 5 9 5 4 1 7 1 7 3 4 8 5 4 3 7 7 5 6 3 7 8 6 0 3 5
 2 3 8 0 0 7 8 3 9 0 0 2 2 9 7 4 3 9 3 1 2 9 1 0 1 3 9 8 9 9 5 5 0 1 5 4 4
 9 2 8 1 6 5 4 6 3 8 3 9 1 2 0 9 1 7 5 7 9 8 9 3 8 0 2 5 7 7 9 8 8 2 5 9 0
 4 7 9 8 7 5]


In [57]:
acc = metrics.accuracy_score(y_test, y_pred)
print("Acuuracy: ", acc)

Acuuracy:  0.9711111111111111
